This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# 影像辨識應用

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
1. `工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`
2. `編輯` -> `筆記本設定` -> 將 `硬體加速器` 改為 `GPU`




## 練習題（一）：有貓就給讚

這世界上有許多人是貓奴，看到貓貓就會給讚。今天我們將從 Reddit 的 [AnimalMemes](https://www.reddit.com/r/AnimalMemes/) 下載梗圖，並且用已經訓練好的 image classification 模型，來找出下載的梗圖中哪些有貓貓。

![](https://i.imgur.com/f4fdLGo.png)

（[圖片來源](https://www.dcard.tw/f/nuu/p/234815577)）

In [ ]:
!pip install matplotlib RedditReader Pillow torch torchvision wget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import wget
from PIL import Image
from RedditReader import Subreddit

### Step 1. 隨機取得 Reddit Meme 的圖片

首先，我們定義一個 function `get_rand_meme()`，這個 function 會隨機的抓取一個 Reddit 上的一個梗圖，並且把梗圖下載到 Colab server 的硬碟裡。

In [ ]:
def get_rand_meme():
    # Download a random meme from Reddit AnimalMemes.
    meme = Subreddit("AnimalMemes")
    meme.get_random()

    # Each meme contains an image URL.
    print(meme.url)

    # Download the image to the disk.
    image_filename = wget.download(meme.url)

    # Return the downloaded image file name.
    return image_filename

我們可以隨機下載一張，並且顯示出來看看。

In [ ]:
image_filename =  get_rand_meme()
print(image_filename)

image = Image.open(image_filename).convert("RGB")
plt.imshow(image)

### Step 2. Image Classification

接下來我們要準備 Image Classification 的模型。不過在開始之前，我們先下載一張貓貓的圖片，方便我們接下的測試。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/cat_lover/cat.jpg -O cat.jpg

In [ ]:
image = Image.open("cat.jpg").convert("RGB")
plt.imshow(image)

我們要用的模型，是在 ImageNet 資料集下所訓練的，總共有 1,000 個 label。由於我們要知道這 1,000 label 對應的英文名稱是什麼，因此我們將對應名稱的檔案下載下來，並且讀進程式以利接下來使用。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/cat_lover/imagenet_labels.json -O imagenet_labels.json

In [ ]:
import json

labels = json.load(open("imagenet_labels.json"))
print(labels)

接下來讀入我們的模型。這裡直接使用 `torchvision` 所提供的由 ImageNet 訓練好的 ResNet-18 模型。

In [ ]:
from torchvision import models

# Get the pre-trained ResNet-18 model.
model = models.resnet18(pretrained=True)
# Move model from CPU to GPU.
model = model.cuda()

# Turn on evaluation mode (no training, inference only).
model.eval()

print(model)

由於模型的輸入必須要固定大小，因此定義了一個轉換的方法：將輸入的圖片轉成固定的大小後，再轉成 PyTorch 的資料格式（`Tensor`），並且根據 ImageNet 的 mean 與 standard deviation 做正規化。

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
    ),
])

這裡定義 function `run_classification()` 來實際做 inference。在將圖片餵給模型之前，除了要做上面提到的轉換，還要將格式從 `CHW` 轉成 `NCHW` 才會對。

In [ ]:
def run_classification(model, transform, labels, image):
    # Apply transformation for the given image.
    input_image = transform(image)
    print(input_image.shape)
    # Convert format from CHW to NCHW.
    input_images = torch.unsqueeze(input_image, 0)
    print(input_images.shape)
    # Move image data from CPU to GPU.
    input_images = input_images.cuda()

    # Do the inference.
    model_out = model(input_images)
    print(model_out.shape)

    # Get the label index that has largest probability.
    label_index = torch.argmax(model_out, 1)[0].item()
    # Get the label.
    label = labels[label_index]

    return label

再來是定義 function `is_cat()` 來根據模型輸出的 label 來判斷是否為貓。由於 ImageNet 的 label 並沒有直接定義是貓的 label，而是各種貓的種類。因此需要多一步來判斷是否為貓。

In [ ]:
def is_cat(label):
    return label in [
        "Egyptian_cat",
        "Persian_cat",
        "tiger_cat",
        "Siamese_cat",
        "tabby",
    ]

完成以上，就可以真的跑模型看看結果囉。

In [ ]:
label = run_classification(model, transform, labels, image)
print(label, is_cat(label))

除此之外，我們也可以上傳自己的圖片到 Colab server，並且做同樣的辨識。（記得將`FILENAME` 改成你的圖片檔名）

In [ ]:
from google.colab import files

files.upload()

In [ ]:
image = Image.open("FILENAME").convert("RGB")

label = run_classification(model, transform, labels, image)
print(label, is_cat(label))

plt.imshow(image)

### Step 3. 兩個功能放一起！

現在我們終於可以試著將上述的兩個功能結合在一起。我們從 Reddit 上下載很多梗圖，並且給模型判斷梗圖的 label，再根據這個 label 來判斷是否為貓。

In [ ]:
num_repeats = 50

cat_labels = []
cat_images = []
for _ in range(num_repeats):
    image_filename =  get_rand_meme()

    image = Image.open(image_filename).convert("RGB")
    label = run_classification(model, transform, labels, image)

    if is_cat(label):
        cat_labels.append(label)
        cat_images.append(image)

最後，我們把所有抓到的貓貓梗圖秀出來吧！

In [ ]:
fig = plt.figure(figsize=(30, 30))
for idx in range(len(cat_labels)):
    ax = fig.add_subplot(
        int(len(cat_labels) / 4) + 1,
        4,
        idx + 1,
        xticks=[],
        yticks=[],
    )
    ax.imshow(cat_images[idx])
    ax.set_title(cat_labels[idx], size=30)

    fig.show()

### 練習時間

1. 將模型從 ResNet-18 改成 [ResNet-152](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet152)，並且試試結果如何
2. 改成 ResNe-152 之後，我們也想想辦法照顧狗奴。請完成以下 function `is_dog()`，並且把上面呼叫 `is_cat()` 的地方改成 `is_dog()`，再跑一次看看結果。

狗狗的 label：
- `dalmatian`
- `Mexican_hairless`
- `Newfoundland`
- `basenji`
- `Leonberg`
- `pug`
- `Great_Pyrenees`
- `Maltese_dog`
- `Old_English_sheepdog`
- `Shetland_sheepdog`
- `Greater_Swiss_Mountain_dog`
- `Bernese_mountain_dog`
- `French_bulldog`
- `Eskimo_dog`
- `African_hunting_dog`


In [ ]:
def is_dog(label):
    return False

## 練習題（二）：貓的追擊者

網路上每天會有許多影片產生，但對於喜歡貓的人而言，他們只要看貓貓就夠了。在這個練習中，我們想要透過 Object Detection 的影像辨識模型，來將有貓貓的影片片段抓取出來。

這次練習的原始影片請看[這個連結](https://pixabay.com/videos/fog-village-dog-cat-gate-41340/)。

![](https://i.imgur.com/NOZOxjH.png)

（[圖片來源](https://goto50.ai/real-time-object-detection-using-tensorflow/)）

In [ ]:
!pip install numpy matplotlib opencv-python Pillow torch torchvision

In [ ]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

### Step 1. 單張影像的 Object Detection

我們先將測試用的圖片下載到 Colab server 的硬碟並且讀取。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/cat_lover/cat.jpg -O cat.jpg

In [ ]:
image = Image.open("cat.jpg").convert("RGB")

plt.imshow(image)

在開始準備模型之前，我們先將模型的 label 定義好。我們用的 Object Detection 模型是用 [COCO](https://cocodataset.org/) 資料集訓練好的，總共有 80 個 label。

In [ ]:
labels = [
    "__background__", "person", "bicycle", "car", "motorcycle", "airplane",
    "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "N/A",
    "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
    "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "N/A", "backpack",
    "umbrella", "N/A", "N/A", "handbag", "tie", "suitcase", "frisbee", "skis",
    "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "N/A", "wine glass",
    "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich",
    "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "N/A", "dining table", "N/A",
    "N/A", "toilet", "N/A", "tv", "laptop", "mouse", "remote", "keyboard",
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "N/A", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
    "toothbrush",
]

這裡我們一樣用 torchvision來建立模型。用的模型架構是 [Faster R-CNN](https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection)，是一個受歡迎的 Object Detection 模型，不過有個缺點是速度相對比較慢。

In [ ]:
from torchvision import models

# Load a Fast R-CNN model pre-trained on COCO.
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Move model from CPU to GPU.
model = model.cuda()

# Turn on evaluation mode (no training, inference only).
model.eval()

關於輸入的轉換，由於這裡的模型輸入不限定影像大小，因此轉換相對簡單。

In [ ]:
from torchvision import transforms

transform = transforms.ToTensor()

接下來定義 function `detect_cat()`，功能是偵測一張圖片中所有貓的 boudning box。流程是將圖片餵給模型，得到 Object Detection 的輸出。由於模型輸出不只有貓的 label，所以必須把貓的 label 篩選出來，並且只留夠夠有信心的結果下來（`score` 大於 `threshold`）。

對於一張圖片來說，Object Detection 模型的輸出為一個 `dict` 物件，包含：
- `labels`：這張圖片中所有偵測到物件的 label index（所以我們還要轉成實際的 label）
- `boxes`：這張圖片中所有偵測到物件的 boudning box
- `scores`：這張圖片中所有偵測到物件的信心值

![](https://i.imgur.com/4Bp9XQk.png)

In [ ]:
def detect_cat(model, transform, image, threshold=0.5):
    # Apply transformation for the given image.
    input_image = transform(image)
    # Convert format from CHW to NCHW.
    input_images = torch.unsqueeze(input_image, 0)
    # Move image data from CPU to GPU.
    input_images = input_images.cuda()

    # Do the inference.
    model_out = model(input_images)
    print(model_out)

    # Get label indexes, bounding boxes and scores from the detection result.
    label_indexes = model_out[0]["labels"]
    boxes = model_out[0]["boxes"]
    scores = model_out[0]["scores"]

    # Filter out the bounding boxes whose labels are "cat" and scores are higher
    # than a threshold.
    cat_boxes = []
    for label_index, box, score in zip(label_indexes, boxes, scores):
        label = labels[label_index]

        if label == "cat" and score >= threshold:
            cat_boxes.append(box)

    return cat_boxes

只要呼叫 `detect_cat()`，就能輕鬆的得到結果了。

In [ ]:
cat_boxes = detect_cat(model, transform, image)

print(cat_boxes)

得到結果後，我們再把結果畫在原本的圖上！

In [ ]:
from PIL import ImageDraw

def draw_boxes(image, boxes):
    image_drawn = image.copy()

    draw = ImageDraw.Draw(image_drawn)
    for box in boxes:
        min_x = int(box[0])
        min_y = int(box[1])
        max_x = int(box[2])
        max_y = int(box[3])

        draw.rectangle(
            [(min_x, min_y), (max_x, max_y)],
            outline ="red",
            width=10,
        )

    return image_drawn

In [ ]:
image_drawn = draw_boxes(image, cat_boxes)
plt.imshow(image_drawn)

也可以上傳你自己的圖片跑跑看！

In [ ]:
from google.colab import files

files.upload()

### Step 2. 影片的 Object Detection

在第二階段，我們將示範如何對影片做 Object Detection。首先下載我們的原始影片。

In [ ]:
!wget https://github.com/SuJiaKuan/fgu_ai_course/raw/main/datasets/cat_lover/cat_dog.mp4

一部影片可以想成是連續好幾張影像組成的，因此我們的思路是將影片轉換成一張一張的影像。這裡用來實現的是 OpenCV 的 [VideoCapture](https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html)，要注意的是由於我們 `detect_cat()` 的影像是 Pillow 的格式，而且影像的顏色順序是 R, G, B，而 OpenCV 讀出來的影像是 `numpy.ndarray`，而且顏色順序是 B, G, R，因此必須要做轉換，才能方便後面的辨識。

In [ ]:
# Create an object to capture frames from a video.
video = cv2.VideoCapture("cat_dog.mp4")

images = []
while True:
    # Capture a frame from the video.
    success, image = video.read()

    # In most case, "success" is True. It becomes False when all frames are
    # captured (or when some errors occur), so we need to break the while loop
    # in such cases.
    if not success:
        break

    # Convert image from OpenCV format (numpy) into Pillow format.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)

    images.append(image)

# Tell the object that the capturing process is finished.
video.release()

print(len(images))
print(images[0].size)

接下來我們將每張擷取出來的影像餵給 Object Detection 模型來辨識，並且將結果的 boudning box 畫出來。

In [ ]:
output_images = []
for image in images:
    cat_boxes = detect_cat(model, transform, image)
    image_drawn = draw_boxes(image, cat_boxes)

    output_images.append(image_drawn)

最後，我們將畫 boudning box 的所有影像輸出成一部影片檔案。這裡用到的是 OpenCV 的 VideoWriter，同樣要注意的是要將 Pillow 影像的格式轉成 OpenCV 的格式，結果才會對。

關於 VideoWriter 的描述如下：
- `fourcc`：輸出影片檔案的編碼（codec）代號
- `fps`：輸出影片檔案的 frame per second，也就是影片的一秒包含幾張影像
- `video_size`：輸出影片檔案的大小

In [ ]:
# Create an object to convert images into a video file.
fourcc = cv2.VideoWriter_fourcc(*"XVID")
fps = 20
video_size = output_images[0].size
out_video = cv2.VideoWriter("output.avi", fourcc, fps, video_size)

for output_image in output_images:
    # Convert image from Pillow format into OpenCV format (numpy).
    image = np.array(output_image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    out_video.write(image)

# Tell the object that the converting process is finished.
out_video.release()

下載影片到你的電腦看看！

In [ ]:
from google.colab import files

files.download("output.avi") 

也可以上傳你自己的影片跑跑看！

In [ ]:
from google.colab import files

files.upload()

### 練習時間

1. 請將偵測貓貓的功能，改成偵測狗狗，並且再跑一次影片看結果
2. 狗狗精華：讓影片只留下有偵測到狗狗的片段，其他的片段都不要，結果類似這個[影片](https://github.com/SuJiaKuan/fgu_ai_course/raw/main/datasets/cat_lover/dog_only.avi)